# Lab 2: Linear-time Robinson-Foulds distance

Implementation in Python of the linear-time algorithm for the Robinson-Foulds distance between two phylogenetic trees.

## About

**Course**  
Bioinformatics and Statistical Genetics (BSG-MIRI)  
FIB - Universitat Politècnica de Catalunya. BarcelonaTech  
Autumn 2021  

**Team**  
* Antoni Casas
&lt;antoni.casas.munoz@estudiantat.upc.edu&gt;
* Marcel Cases
&lt;marcel.cases@estudiantat.upc.edu&gt;

## The algorithm

We use `ete3` as the library for parsing phylogenetic trees from a string.  
We use the Newick representation.

In [1]:
from ete3 import Tree

# Loads a tree structure from a newick string. The returned variable ’t’ is the root node for the tree.
t = Tree("((((1,2),3),4),9);" )
t2 = Tree("(8,(1,(2,(3,4))));" )

The input trees that we want to use for calculating the distance with the Robinson-Foulds algorithm are the following two:

In [2]:
print("t:", t)
print("t2:", t2)

t: 
            /-1
         /-|
      /-|   \-2
     |  |
   /-|   \-3
  |  |
--|   \-4
  |
   \-9
t2: 
   /-8
--|
  |   /-1
   \-|
     |   /-2
      \-|
        |   /-3
         \-|
            \-4


In [3]:
def reverseTraversalT1(node, resultSet,numClust):
    numClust=numClust+1
    if len(node.children)==0:
        newLabel=len(translationTable)
        translationTable[node.name]=newLabel
        currentClust=(newLabel,newLabel)
    else:
        minval=10^9
        maxval=0
        numChildren=len(node.children)
        for idx,child in enumerate(node.children):
            (leafClust,numClust)=reverseTraversalT1(child,resultSet,numClust)
            maxval=max(maxval,leafClust[1])
            minval=min(minval,leafClust[0])
            if (idx==numChildren-1):
                pseudoHash[leafClust[0]]=leafClust[1]
            else:
                pseudoHash[leafClust[1]]=leafClust[0]
        currentClust=(minval,maxval)
    resultSet.append(currentClust)
    return (currentClust,numClust)

In [4]:
def reverseTraversalT2(node, resultSet, numClust):
    numClust=numClust+1
    currentClust=None
    if len(node.children)==0:
        if (node.name in translationTable):
            currentClust=(translationTable[node.name],translationTable[node.name],1)
            resultSet.append(currentClust)
            visited[translationTable[node.name]]=True
        else:
            visited[len(translationTable)]=False
            currentClust=(-1000,-1000,1)
            resultSet.append(currentClust)
    else:
        minval=10^9
        maxval=0
        elements=0
        numChildren=len(node.children)
        for idx,child in enumerate(node.children):
            (leafClust,numClust)=reverseTraversalT2(child,resultSet,numClust)
            if (leafClust is not None):
                maxval=max(maxval,leafClust[1])
                minval=min(minval,leafClust[0])
                elements=elements+leafClust[2]
                theoreticalElems=leafClust[1]-leafClust[0]
                trueElems=leafClust[2]
                if theoreticalElems==(trueElems-1) and theoreticalElems>0:
                    if (idx==numChildren-1) and pseudoHash[leafClust[0]]==leafClust[1]:
                        resultSet.append(leafClust)
                    elif pseudoHash[leafClust[1]]==leafClust[0]:
                        resultSet.append(leafClust)
        currentClust=(minval,maxval,elements)
    return (currentClust,numClust)

In [5]:
translationTable={}
pseudoHash=[0]*len(t)
numClust=0
visited=[False]*len(t)+[True]

auxiliar=[]
(useless,numClust)=reverseTraversalT1(t,auxiliar,numClust)


auxiliar2=[]
(why,numClust)=reverseTraversalT2(t2,auxiliar2,numClust)


trivial=1
for elem in visited:
    if elem==False:
        trivial=0
        break

The normalised Robinson-Foulds distance is:

In [6]:
(numClust-(len(auxiliar2)+trivial)*2)/numClust

0.3333333333333333